In [1]:
!pip install haversine

In [2]:
import pandas as pd
import numpy as np
import re
import haversine as hs
import folium
import datetime

from haversine import Unit


In [4]:
pubs_data = pd.read_csv("/content/data-wrangling-pubs-full-clean.csv", index_col=[0])
police_metro_data = pd.read_csv(
    "/content/data-wrangling-police_metro_clean.csv", index_col=[0]
)
underground_data = pd.read_csv("/content/raw-london-underground.csv")
crime_data = pd.read_csv("/content/raw_london_sexual_crimes.csv", index_col=[0])


In [5]:
police_stations = police_metro_data[police_metro_data["category"] == "Police Station"]


In [6]:
pubs_data.head()

,fsq_id,name,address,locality,neighborhood,postcode,category,latitude,longitude,popularity,price,rating
0,527c07e711d255a8e45a3856,The Aeronaut,264 Acton High St,Acton,Acton,W3 9BH,Pub,51.508508,-0.275519,0.954860,2.0,7.6
1,4db89f1dcda10f31e61b8ac9,The Talbot Inn,1 Mill Hill Rd,Greater London,Acton,W3 8JB,Pub,51.506495,-0.273439,0.880282,1.0,6.6
2,55d77b39498e56569779feb2,The Sheepwalk Tavern,14 Market Pl,London,Acton,W3 6QS,Pub,51.507626,-0.271289,0.893734,1.0,NaN
3,4bf462f12d629521a21d6058,Duke of York,86 Steyne Rd,Acton,Acton,W3 9NU,Night Club,51.509316,-0.272190,0.878483,1.0,NaN
4,4bd5ff6c6f649521f71970ec,The Belvedere,108-112 High St,London,Acton,W3 6QX,Pub,51.506873,-0.268905,0.876107,1.0,5.9


In [7]:
underground_data.head()

,Unnamed: 0,name,latitude,longitude,line
0,0,Acton Town,51.502500,-0.278126,"District, Piccadilly"
1,1,Acton Central,51.508835,-0.263033,London Overground
2,2,Acton Central,51.508560,-0.262880,London Overground
3,3,Aldgate,51.513940,-0.075370,Metropolitan
4,4,Aldgate East,51.515140,-0.071780,"District, Hammersmith & City"


In [8]:
police_stations.head()

,fsq_id,name,address,locality,neighborhood,postcode,category,latitude,longitude
0,4d6e854c2e6ea09398fe1e50,Acton Police Station,250 High St,Acton,Acton,W3 9BH,Police Station,51.508535,-0.274953
1,4c99b383eaa5a143b7eec5e4,Bishopsgate Police Station,182 Bishopsgate,London,Spitalfields,EC2M 4NP,Police Station,51.517620,-0.080110
2,838a9a92b3244f53520eb6b7,British Transport Police,"Bishopsgate Police Station, 182 Bishopsgate",London,Whitechapel,EC2M 4NP,Police Station,51.517735,-0.079948
4,6118a7a7533844f4df964ca3,The Civil Justice Council,NaN,London,NaN,WC2A 2LL,Police Station,51.513601,-0.114607
5,d045b823812e486e28bc8834,Calder UK,2 Arundel St,London,NaN,WC2R 3AZ,Police Station,51.511783,-0.114156


In [11]:
crime_data.head()

,crime_id,month,reported_by,falls_within,longitude,latitude,location,LSOA code,LSOA name,crime_type,Last outcome category,Context
1,3f8f7ac887eaf9211b0ae601304b26722a8d19bbb5ba1b...,2021-12,City of London Police,City of London Police,-0.109640,51.520624,On or near Leather Lane,E01000916,Camden 027B,Violence and sexual offences,Under investigation,NaN
11,3f8f7ac887eaf9211b0ae601304b26722a8d19bbb5ba1b...,2021-12,City of London Police,City of London Police,-0.097562,51.518864,On or near Parking Area,E01000001,City of London 001A,Violence and sexual offences,Unable to prosecute suspect,NaN
21,fcd78bb549a08c5f2cbfbb0c70b6ba8e5dd61fa8ebe1b9...,2021-12,City of London Police,City of London Police,-0.095026,51.518499,On or near Park/Open Space,E01000002,City of London 001B,Violence and sexual offences,Under investigation,NaN
22,f4afa08625bdc8161e9dc0b14b39049581e5bc3a643e6f...,2021-12,City of London Police,City of London Police,-0.092628,51.517920,On or near St Alphage Garden,E01000002,City of London 001B,Violence and sexual offences,Under investigation,NaN
23,419176d79450dc395d689eed75c05ec59b6e84b7d43159...,2021-12,City of London Police,City of London Police,-0.092723,51.517032,On or near Aldermanbury Square,E01000002,City of London 001B,Violence and sexual offences,Under investigation,NaN


In [10]:
crime_data.rename(
    {
        "Crime ID": "crime_id",
        "Month": "month",
        "Reported by": "reported_by",
        "Falls within": "falls_within",
        "Longitude": "longitude",
        "Latitude": "latitude",
        "Location": "location",
        "Crime type": "crime_type",
    },
    axis=1,
    inplace=True,
)


In [12]:
def get_distance(latitude1, longitude1, df):
    loc1 = (latitude1, longitude1)
    distance_min = 1000000
    for latitude2, longitude2 in list(zip(df.latitude, df.longitude)):
        loc2 = (latitude2, longitude2)
        distance = hs.haversine(loc1, loc2, unit=Unit.METERS)
        if distance < distance_min:
            distance_min = distance
    return distance_min


In [13]:
pubs_data["distance_metro"] = pubs_data.apply(
    lambda row: get_distance(row["latitude"], row["longitude"], underground_data),
    axis=1,
)


In [14]:
pubs_data["distance_police"] = pubs_data.apply(
    lambda row: get_distance(row["latitude"], row["longitude"], police_stations), axis=1
)


In [17]:
pubs_data.sort_values("distance_metro", ascending=False).head(10)


,fsq_id,name,address,locality,neighborhood,postcode,category,latitude,longitude,popularity,price,rating,distance_metro,distance_police
883,4aecb4f2f964a52072ca21e3,The Manor Arms,Wades Pl,London,Canary Wharf,NaN,Pub,46.314475,11.048029,0.912509,1.0,NaN,979179.373436,983716.420580
926,57079e57498e59fb6567f3d9,The Thirsty Scarecrow,107 High St,Dover,Dover,CT16 1EB,Pub,51.130003,1.305305,0.984743,1.0,NaN,87363.955968,94176.472235
925,4e3b09c42271d21e86d6643a,The Eagle Inn,324 London Rd,Dover,Dover,CT17 0SX,Pub,51.130413,1.304748,0.958391,1.0,NaN,87306.375889,94122.980375
924,4c0822bdbbc676b0241146d5,The Red Lion,54 Charlton Green,Dover,Dover,CT16 2PS,Pub,51.132618,1.306131,0.876560,1.0,NaN,87251.834164,94117.392808
927,4e8f3a2fb63456ab94f00585,The Old Endeavour,124 London Rd,Dover,Dover,CT17 0TG,Pub,51.135698,1.297483,0.752294,1.0,NaN,86559.513987,93428.716038
1127,4c09f1aded259521a70c1356,The Wheatsheaf,135 High St,West Wickham,West Wickham,BR4 0LU,Pub,51.376910,-0.020849,0.849325,1.0,5.0,11085.598186,402.995018
234,4c016bbff7ab0f47301616b6,The Compass,10 Widmore Rd,Bromley,Bromley,BR1 1RY,Pub,51.406154,0.016200,0.859944,1.0,NaN,8645.505501,744.318193
231,4bdde498ffdec92807dbe6a1,O'Neill's,27-29 East St,Bromley,Bromley,BR1 1QE,Pub,51.406950,0.015863,0.948544,NaN,7.6,8555.063249,834.833277
232,4e2890dc52b1c37d35c3660b,The Greyhound,205 High St,Bromley,Bromley,BR1 1NY,Pub,51.406398,0.013243,0.976999,1.0,7.0,8540.199188,816.826089
811,5c275f85e679bc002ca7f0bc,Pope Street Bar & Kitchen,396 Footscray Rd,London,NaN,SE9 3TU,Pub,51.437303,0.072182,0.965351,1.0,NaN,8487.798068,2093.932850


In [20]:
pubs_data.sort_values("distance_police", ascending=False).head(10)


,fsq_id,name,address,locality,neighborhood,postcode,category,latitude,longitude,popularity,price,rating,distance_metro,distance_police
883,4aecb4f2f964a52072ca21e3,The Manor Arms,Wades Pl,London,Canary Wharf,NaN,Pub,46.314475,11.048029,0.912509,1.0,NaN,979179.373436,983716.420580
926,57079e57498e59fb6567f3d9,The Thirsty Scarecrow,107 High St,Dover,Dover,CT16 1EB,Pub,51.130003,1.305305,0.984743,1.0,NaN,87363.955968,94176.472235
925,4e3b09c42271d21e86d6643a,The Eagle Inn,324 London Rd,Dover,Dover,CT17 0SX,Pub,51.130413,1.304748,0.958391,1.0,NaN,87306.375889,94122.980375
924,4c0822bdbbc676b0241146d5,The Red Lion,54 Charlton Green,Dover,Dover,CT16 2PS,Pub,51.132618,1.306131,0.876560,1.0,NaN,87251.834164,94117.392808
927,4e8f3a2fb63456ab94f00585,The Old Endeavour,124 London Rd,Dover,Dover,CT17 0TG,Pub,51.135698,1.297483,0.752294,1.0,NaN,86559.513987,93428.716038
225,539a0fdb498ef219412fe560,The Scotsman,96 Scotts Rd,Southall,Southall,UB2 5DE,Pub,51.502110,-0.394197,0.104056,1.0,NaN,3205.230302,5901.038828
1019,4ead75f0f7906795c3b30f23,Innisfree Pub,NaN,London,NaN,NaN,Pub,51.546125,-0.273271,0.913979,1.0,NaN,299.414361,4181.442372
1160,4b645a2af964a520a3ac2ae3,Railway Tavern,126 Snakes Lane East,London,Woodford Green,IG8 7HY,Pub,51.607729,0.039159,0.947794,1.0,NaN,458.137473,3629.761719
1154,4ba295caf964a520a60638e3,The Salisbury Arms,Hoppers Rd,London,Winchmore Hill,N21 3NP,Pub,51.633010,-0.103004,0.971911,1.0,7.0,1702.389734,3318.668170
603,4ac518c7f964a52060a520e3,Grand Junction Arms,Acton Lane,London,Willesden,NW10 7AD,Pub,51.534390,-0.260489,0.971389,1.0,8.1,289.148378,3044.141983


In [21]:
# drop rows from other city
pubs_data.drop([883, 926, 925, 924, 927], axis=0, inplace=True)


In [22]:
def get_number_of_crimes(latitude1, longitude1, df):
    loc1 = (latitude1, longitude1)
    crimes = 0
    for latitude2, longitude2 in list(zip(df["latitude"], df["longitude"])):
        loc2 = (latitude2, longitude2)
        distance = hs.haversine(loc1, loc2, unit=Unit.METERS)
        if distance <= 500:
            crimes += 1
    return crimes


In [23]:
pubs_data["number_of_crimes"] = pubs_data.apply(
    lambda row: get_number_of_crimes(row["latitude"], row["longitude"], crime_data),
    axis=1,
)


In [24]:
pubs_data.head()

,fsq_id,name,address,locality,neighborhood,postcode,category,latitude,longitude,popularity,price,rating,distance_metro,distance_police,number_of_crimes
0,527c07e711d255a8e45a3856,The Aeronaut,264 Acton High St,Acton,Acton,W3 9BH,Pub,51.508508,-0.275519,0.954860,2.0,7.6,691.998156,39.286398,145
1,4db89f1dcda10f31e61b8ac9,The Talbot Inn,1 Mill Hill Rd,Greater London,Acton,W3 8JB,Pub,51.506495,-0.273439,0.880282,1.0,6.6,550.067018,249.869699,162
2,55d77b39498e56569779feb2,The Sheepwalk Tavern,14 Market Pl,London,Acton,W3 6QS,Pub,51.507626,-0.271289,0.893734,1.0,NaN,586.979868,272.981363,174
3,4bf462f12d629521a21d6058,Duke of York,86 Steyne Rd,Acton,Acton,W3 9NU,Night Club,51.509316,-0.272190,0.878483,1.0,NaN,635.964569,210.015382,156
4,4bd5ff6c6f649521f71970ec,The Belvedere,108-112 High St,London,Acton,W3 6QX,Pub,51.506873,-0.268905,0.876107,1.0,5.9,457.270081,457.557147,180


In [26]:
pubs_data.to_csv("data-wrangling-pubs_data_with_crimes.csv")